In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from utils.dataset import get_dataloaders
from utils.metrics import evaluate_model  # Ensure it returns: report, accuracy, matrix

from models.vgg import CustomVGG
from models.resnet import CustomResNet
from models.mobilenet import CustomMobileNet
from models.inception import CustomInception
from models.densenet import CustomDenseNet

# Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

In [2]:
data_dir = r'Pets\Master Folder'
train_loader, val_loader, class_names = get_dataloaders(data_dir)
num_classes = len(class_names)


In [3]:
model_map = {
    'vgg': CustomVGG,
    'resnet': CustomResNet,
    'mobilenet': CustomMobileNet,
    'inception': CustomInception,
    'densenet': CustomDenseNet
}



In [4]:
def train_and_evaluate_model(model_name, model_class, num_classes, device, 
                             train_loader, val_loader, class_names, epochs=10, lr=1e-3):
    print(f"\n==== Training {model_name.upper()} model ====")
    model = model_class(num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    best_acc = 0
    train_losses = []
    val_accuracies = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        avg_loss = running_loss / len(train_loader)
        train_losses.append(avg_loss)

        print(f"\nEpoch [{epoch+1}/{epochs}] - Training Loss: {avg_loss:.4f}")

        # Evaluation
        report, accuracy, matrix = evaluate_model(model, val_loader, device, class_names)
        val_accuracies.append(accuracy)

        print("\nValidation Classification Report:\n", report)
        print("\nConfusion Matrix:")
        print(matrix)

        # Plot confusion matrix for each epoch
        plot_confusion_matrix(matrix, class_names, f"{model_name} - Epoch {epoch+1}")

        # Save best model
        if accuracy > best_acc:
            best_acc = accuracy
            torch.save(model.state_dict(), f'best_model_{model_name}.pth')
            print(f"New best model saved (Accuracy: {best_acc:.4f})")

        scheduler.step()

    # Plot final training loss and accuracy
    plot_training_curves(train_losses, val_accuracies, model_name)

    # Print final accuracy
    print(f"\nTraining complete. Best validation accuracy: {best_acc:.4f}\n")

    del model
    torch.cuda.empty_cache()
    return best_acc


def plot_confusion_matrix(cm, class_names, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix - {title}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.show()


def plot_training_curves(train_losses, val_accuracies, model_name):
    epochs_range = range(1, len(train_losses) + 1)
    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, train_losses, marker='o', color='blue', label='Training Loss')
    plt.title(f"{model_name.upper()} - Training Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, val_accuracies, marker='x', color='green', label='Validation Accuracy')
    plt.title(f"{model_name.upper()} - Validation Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.ylim(0, 1)
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
results = {}
best_model_name = None
best_accuracy = 0.0

for name, model_class in model_map.items():
    acc = train_and_evaluate_model(
        model_name=name,
        model_class=model_class,
        num_classes=num_classes,
        device=device,
        train_loader=train_loader,
        val_loader=val_loader,
        class_names=class_names,
        epochs=10,
        lr=1e-3
    )
    results[name] = acc
    if acc > best_accuracy:
        best_accuracy = acc
        best_model_name = name

print(f"\n Best model is: {best_model_name.upper()} with Accuracy = {best_accuracy:.4f}")
with open("best_models_log.txt", "a") as log_file:
        log_file.write(f"{best_model_name},{best_accuracy:.4f}\n")


==== Training VGG model ====
